# 本地缓存文件

`fsspec` 允许你访问远程文件系统上的数据，这是它的目的。然而，与本地存储相比，这种访问通常相当慢，因此除了缓冲（见上文）之外，还有选项在你第一次访问时将文件复制到本地，并在此后使用本地数据。这些本地缓存的数据可能是临时的（即，附加到进程并在进程结束时丢弃），或者位于本地存储中的某个特定位置。

提供了两种机制，都涉及包装 `target` 文件系统。以下示例创建了一个基于文件的缓存。

```python
fs = fsspec.filesystem("filecache", target_protocol='s3', target_options={'anon': True},
                       cache_storage='/tmp/files/')
```

每次你在 S3 上打开一个远程文件，它都会首先将其复制到本地临时目录，然后所有进一步的访问都将使用本地文件。由于我们指定了特定的本地位置，文件将持久化并且可以在未来的会话中重用，尽管你也可以设置策略让缓存文件在一定时间后过期，或者在每次打开时检查远程文件系统，看看目标文件自从被复制以来是否有变化。

使用顶级函数 `open`、`open_local` 和 `open_files`，你可以使用与上面示例相同的一组 `kwargs`，或者你可以链接 URL - 以下将是等效的：

```python
of = fsspec.open("filecache::s3://bucket/key",
                 s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})
```                 

使用“blockcache”变体，数据会分块下载：只访问远程文件的特定部分。这意味着，如果只需要文件的某些部分，本地副本的大小可能比远程文件小得多。

尽管“filecache”适用于所有文件系统实现，并为其他库提供了一个真实的本地文件，但“blockcache”有限制：你必须有一个支持稀疏文件的存储/操作系统组合，后端实现使用 `from AbstractBufferedFile` 的文件，你传递给结果对象的库接受通用的 python 类文件对象。你不应该在同一个目录中混合使用块缓存和文件缓存。“simplecache”与“filecache”相同，只是没有缓存过期和检查原始源的选项 - 它可以在目标被认为是静态的情况下使用，特别是在预期有大量的目标文件时（因为不向磁盘写入元数据）。只有“simplecache”保证线程/进程安全。